In [111]:
import numpy as np
import pandas as pd
from itertools import product

import gurobipy as grb
from gurobipy import GRB

# Input Problem Parameters

## Définissons les variables

In [112]:
from extract_parameters import extract_parameters

################################
######## TO DO #################
################################

# choose taille among: "small", "medium", "large"
taille = "small"

In [113]:
################################
######## DO NOT CHANGE #########
################################
conge, qualif, d, p, b, besoin = extract_parameters(taille)

In [114]:
print("conge : ",conge)
print("qualif : ",qualif)
print("date livraison : ",d)
print("pénalité : ",p)
print("bénéfice : ",b)
print("besoin : ",besoin)

conge :  [[1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 1, 1, 1, 1]]
qualif :  [[0, 1, 1], [0, 1, 1], [1, 0, 1]]
date livraison :  [3, 3, 4, 3, 5]
pénalité :  [3, 3, 3, 3, 3]
bénéfice :  [20, 15, 15, 20, 10]
besoin :  [[1, 1, 1, 0, 0], [1, 2, 0, 2, 0], [1, 0, 2, 1, 2]]


## Définissons les paramètres

In [115]:
from define_nums import define_nums

nombre_employes, horizon, nombre_qualif, nombre_projets = define_nums(conge, qualif, d, p, b, besoin)

In [116]:
print("nombre_employes : ",nombre_employes)
print("horizon : ",horizon)
print("nombre_qualif : ",nombre_qualif)
print("nombre_projets : ",nombre_projets)

nombre_employes :  3
horizon :  5
nombre_qualif :  3
nombre_projets :  5


In [117]:
## u_j, le projet j est réalisé
u = [0]*nombre_projets
u_shape = len(u)

## x_i_j_k_n, l'employé k travaille sur le projet j avec la qualif i
## au jour n
x = [[[[0]*horizon for _ in range(nombre_employes)] for _ in range(nombre_projets)] for _ in range(nombre_qualif)]
x_shape = [(i,j,k, n) for i in range(nombre_qualif) for j in range(nombre_projets) for k in range(nombre_employes) for n in range(horizon)]

## t_k_j, l'employé k travaille sur le projet j
t = [[0]*nombre_projets for _ in range(nombre_employes)]
t_shape = [(i,j) for i in range(nombre_employes) for j in range(nombre_projets)]

## begin_j, la date de début effective
begin = [0]*nombre_projets
begin_shape = len(begin)

## end_j, la date de fin effective = date de livraison du projet j
end = [0]*nombre_projets
end_shape = len(end)

## delay_j, le nombre de jours de retard par projet j
delay = [0]*nombre_projets

## longueur_j, la longueur du projet j
longueur = [0]*nombre_projets

## projet_par_employe_k, le nombre de projet par employe
projet_par_employe = [0]*nombre_employes

# majorant pour la contrainte de couverture
maj = horizon * nombre_employes

# Model deployment

## Add the variables

In [102]:
model = grb.Model("Planning")

# L'affectation du planning, la variable clé
x = model.addVars(x_shape, vtype=GRB.BINARY, name="x")

## Les dates de fin des projets
dateFin = model.addVars(end_shape, lb = 0, ub = horizon, name = "dateFin")

## Les dates de début des projets
dateDebut = model.addVars(begin_shape, lb = 0, ub = horizon, name = "dateDebut")

## L'affectation sur un projet d'un employé
affectation = model.addVars(t_shape, vtype = GRB.BINARY, name = "affectation")

## La réalisation ou non d'un projet
u = model.addVars(u_shape, vtype = GRB.BINARY, name="u")

In [103]:
# Update le modele pour confirmer la création des variables
model.update()


## Add constraints

In [104]:
for i in range(nombre_qualif):
    for j in range(nombre_projets):
        for k in range(nombre_employes):
            for n in range(horizon):
                ## La qualification du personnel
                print(i,j,k,n)
                model.addConstr( x[i,j,k,n] <= qualif[i][k], name="QualificationPersonnel[{0}{1}{2}{3}]".format(i,j,k,n))

                ## Contraintes des congés
                model.addConstr( x[i,j,k,n] <= conge[k][n], name=f"CongesPersonnel[{i}{j}{k}{n}]")
        
        ## Contrainte d'unicité de réalisation d'un projet
        #model.addConstr( grb.quicksum(grb.quicksum(x[i,j,k,n] for n in range(horizon)) for k in range(nombre_employes)) >= besoin[i][j]*u[j], name=f"UniciteRealisationMin[{i}{j}]")
        model.addConstr( grb.quicksum(grb.quicksum(x[i,j,k,n] for n in range(horizon)) for k in range(nombre_employes)) <= besoin[i][j], name=f"UniciteRealisation[{i}{j}]")

        ## Couverture des qualifications
        model.addConstr( grb.quicksum(grb.quicksum(x[i,j,k,n] for n in range(horizon)) for k in range(nombre_employes)) >= maj*(1-u[j]), name=f"CouvertureQualificationsInf[{i}{j}]")
        model.addConstr( grb.quicksum(grb.quicksum(x[i,j,k,n] for n in range(horizon)) for k in range(nombre_employes)) <= maj*u[j] -10**-4, name=f"CouvertureQualificationsSup[{i}{j}]")


## Unicité de l'affectation quotidienne
for k in range(nombre_employes):
    for n in range(horizon):
        model.addConstr( grb.quicksum(grb.quicksum(grb.quicksum(grb.quicksum(x[i,j,kp,np]*(np==n)*(kp==k) for np in range(horizon)) for kp in range(nombre_employes)) for j in range(nombre_projets)) for i in range(nombre_qualif)) <= 1, name=f"UniciteAffectation[{k}{n}]" )


for j in range(nombre_projets):
    ## end date
    model.addConstrs(((x[i,j,k,n] * n <= dateFin[j]) for i in range(nombre_qualif) for k in range(nombre_employes) for n in range(horizon)), name=f"FinProjet[{j}]")

    ## longueur
    longueur[j] = dateFin[j] - begin[j]

    ## delay
    delay[j] = max(0, dateFin[j] - d[j])



## projet_par_employe
for k in range(nombre_employes):
    projet_par_employe[k] = grb.quicksum( affectation[k,j] for j in range(nombre_projets))

0 0 0 0
0 0 0 1
0 0 0 2
0 0 0 3
0 0 0 4
0 0 1 0
0 0 1 1
0 0 1 2
0 0 1 3
0 0 1 4
0 0 2 0
0 0 2 1
0 0 2 2
0 0 2 3
0 0 2 4
0 1 0 0
0 1 0 1
0 1 0 2
0 1 0 3
0 1 0 4
0 1 1 0
0 1 1 1
0 1 1 2
0 1 1 3
0 1 1 4
0 1 2 0
0 1 2 1
0 1 2 2
0 1 2 3
0 1 2 4
0 2 0 0
0 2 0 1
0 2 0 2
0 2 0 3
0 2 0 4
0 2 1 0
0 2 1 1
0 2 1 2
0 2 1 3
0 2 1 4
0 2 2 0
0 2 2 1
0 2 2 2
0 2 2 3
0 2 2 4
0 3 0 0
0 3 0 1
0 3 0 2
0 3 0 3
0 3 0 4
0 3 1 0
0 3 1 1
0 3 1 2
0 3 1 3
0 3 1 4
0 3 2 0
0 3 2 1
0 3 2 2
0 3 2 3
0 3 2 4
0 4 0 0
0 4 0 1
0 4 0 2
0 4 0 3
0 4 0 4
0 4 1 0
0 4 1 1
0 4 1 2
0 4 1 3
0 4 1 4
0 4 2 0
0 4 2 1
0 4 2 2
0 4 2 3
0 4 2 4
1 0 0 0
1 0 0 1
1 0 0 2
1 0 0 3
1 0 0 4
1 0 1 0
1 0 1 1
1 0 1 2
1 0 1 3
1 0 1 4
1 0 2 0
1 0 2 1
1 0 2 2
1 0 2 3
1 0 2 4
1 1 0 0
1 1 0 1
1 1 0 2
1 1 0 3
1 1 0 4
1 1 1 0
1 1 1 1
1 1 1 2
1 1 1 3
1 1 1 4
1 1 2 0
1 1 2 1
1 1 2 2
1 1 2 3
1 1 2 4
1 2 0 0
1 2 0 1
1 2 0 2
1 2 0 3
1 2 0 4
1 2 1 0
1 2 1 1
1 2 1 2
1 2 1 3
1 2 1 4
1 2 2 0
1 2 2 1
1 2 2 2
1 2 2 3
1 2 2 4
1 3 0 0
1 3 0 1
1 3 0 2
1 3 0 3
1 3 0 4


NotImplementedError: 

## Fonctions objectif

In [105]:
## bénéfice total
benefice_tot = grb.quicksum( (b[j] - delay[j]*p[j])*u[j] for j in range(nombre_projets) )

## Nombre de projet maximum par personne
projet_max = model.addVar(name="projet_max")
model.addConstrs(projet_max>=projet_par_employe[k] for k in range(nombre_employes))
model.update()

## Compacité
longueur_max = grb.max_(longueur)

model.ModelSense = GRB.MINIMIZE
model.setObjectiveN(benefice_tot, 0, 2)
#model.setObjectiveN(projet_max, 1, 1)
#model.setObjectiveN(longueur_max, 2, 0)

In [106]:
### Corriger les fonctions obj 2 et 3 qui plantent
### Comprendre pq on passe jamais nos x à 1

# Model optimisation

In [107]:
# Verify model formulation

model.write('marketSharing.lp')

# Run optimization engine

model.optimize()

# Print the values of all variables
for v in model.getVars():
    print(f"{v.VarName} = {v.X}")

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 558 rows, 251 columns and 1464 nonzeros
Model fingerprint: 0xebdff5d5
Variable types: 6 continuous, 245 integer (245 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e-04, 2e+01]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 1 objectives ... 
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 () ...
---------------------------------------------------------------------------

Optimize a model with 558 rows, 251 columns and 1464 nonzeros
Model fingerprint: 0x62

In [81]:
model.objVal

80.0

In [67]:
x_values = [var.x for var in model.getVars()]
print(x_values)
print(horizon*nombre_projets*nombre_employes*nombre_qualif)
print(nombre_projets*(2+nombre_employes))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,